# **Imports**

In [1]:
# IMPORTS
import numpy as np
import cv2
import os
import sys
import random
import matplotlib.pyplot as plt
import datetime
import pickle as pkl
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import layers, regularizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from scipy import misc, ndimage
from PIL import Image
from keras.optimizers import Adam, RMSprop
import time
from IPython.display import clear_output
from keras.backend import sigmoid
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))
from keras.utils.generic_utils import get_custom_objects
from tensorflow.keras.layers import Activation
get_custom_objects().update({'swish': Activation(swish)})
from keras.regularizers import l2, l1, l1_l2
from keras.backend import manual_variable_initialization 
manual_variable_initialization(True)
from sklearn.metrics import classification_report, confusion_matrix
import requests
import json
import glob
from math import exp
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, RemoteMonitor, LambdaCallback, ReduceLROnPlateau, Callback, LearningRateScheduler
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mimg
%matplotlib inline
plt.rcParams["figure.figsize"] = (10,7)
import matplotlib.backends.backend_pdf
import pandas as pd
from scipy import misc
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import random
from sklearn.model_selection import KFold
from numpy import savetxt
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
from custom_callbacks import GetLearningRate
import tensorflow.keras.backend as K

Using TensorFlow backend.


In [ ]:
# DEFINE FUNCTIONS AND CLASSES
def tg_sendtext(bot_message):
    try:
        bot_token = '1483800645:AAHybxYA3hLnwhegjU4wV6oeNpFPtfOcuh4'
        bot_chatID = '1220107026'
        send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message

        response = requests.get(send_text)
        return response.json()
    except:
        print('Message Failed To Send')
        pass

def message_discord(message):
    try:
        channelID = "777064829063135234" # enable dev mode on discord, right-click on the channel, copy ID
        botToken = "Nzc3MDYzODcyMjgzNTQxNTE0.X69_IA.TtsQh9eVhxFfeakGkQLqn9UDAxw"    # get from the bot page. must be a bot, not a discord app

        baseURL = "https://discordapp.com/api/channels/{}/messages".format(channelID)
        headers = { "Authorization":"Bot {}".format(botToken),
                    "User-Agent":"myBotThing (http://some.url, v0.1)",
                    "Content-Type":"application/json", }

        POSTedJSON =  json.dumps ( {"content":message} )

        r = requests.post(baseURL, headers = headers, data = POSTedJSON)
    except:
        print('Message Failed To Send')
        pass

def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

def check_accuracy(mdl, X_new, y_new):
    true = 0
    Y_pred = mdl.predict(X_new)
    y_pred = np.argmax(Y_pred, axis=1)
    for j in range(len(y_new)):
        if y_pred[j] == y_new[j]:
            true += 1
    accc = (true/len(y_new))*100
    return accc

def check_highest(folder_path1, train_max, max_acc, max_i):
    temp_max_acc = [0]
    temp_max_i = [0]
    for i in os.listdir(folder_path1):
        model = load_model(folder_path1 + i, custom_objects = {"Activation": Activation, "lr": lr})
        acc = check_accuracy(model, X_new, y_new)
        print('Model Accuracy =', str(acc) + '%')
        if acc > 97.4:
            os.remove(folder_path1 + i)
            new_name = i.replace(".h5","") + '_' + str(round(acc, 2)) + '.h5'
            model.save('good_models/' + new_name)
            temp_max_acc.append(acc)
            temp_max_i.append(new_name)
        else:
            os.remove(folder_path1 + i)
    max_acc = np.append(max_acc, temp_max_acc)
    max_i = np.append(max_i, temp_max_i)
    maxx = np.amax(max_acc)
    index = np.where(max_acc == maxx)
    if maxx > train_max:
        train_max = maxx
        print('New High Accuracy =', train_max)
        mess = str(train_max)
        tg_sendtext('New High Accuracy = ' + mess)
        message_discord('New High Accuracy = ' + mess)
    print('The highest Accuracy:', max_i[index], 'with', str(maxx) + '%')
    return train_max, acc, max_acc, max_i

def check_highest_custom(folder_path1, folder_path2, val):
    for i in os.listdir(folder_path1):
        model = load_model(folder_path1 + i, custom_objects = {"Activation": Activation, "lr": lr})
        acc = check_accuracy(model, X_new, y_new)
        if acc > val:
            os.remove(folder_path1 + i)
            new_name = str(round((random.uniform(0, 10) + random.uniform(0, 10)) * random.uniform(0, 100), 5)) + '_' + i.replace(".h5","") + '.h5'
            model.save(folder_path2 + new_name)
        else:
            os.remove(folder_path1 + i)
            
def check_highest_rename(folder_path1, folder_path2, val, X_new, Y_new):
    for i in os.listdir(folder_path1):
        model = load_model(folder_path1 + i, custom_objects = {"Activation": Activation, "lr": lr})
        acc = check_accuracy(model, X_new, Y_new)
        print('Model Name:', i)
        print('Model Accuracy =', str(acc) + '%')
        print('------------------------------------------------------------------------')
        if acc > val:
            os.remove(folder_path1 + i)
            new_name = str(round((random.uniform(0, 10) + random.uniform(0, 10)) * random.uniform(0, 100), 5)) + '_' +i.replace(".h5","") + '_' + str(round(acc, 2)) + '.h5'
            model.save(folder_path2 + new_name)
        else:
            os.remove(folder_path1 + i)

def data():
    
    class GetLearningRate(Callback):
        def __init__(self, array):
            self.array = array

        def on_epoch_end(self, epoch, logs=None):
            lr = self.model.optimizer.lr
            decay = self.model.optimizer.decay
            iterations = self.model.optimizer.iterations
            lr_with_decay = lr / (1. + decay * K.cast(iterations, K.dtype(decay)))
            self.array.append(lr_with_decay)
            print('Learning Rate = ', K.eval(lr_with_decay))
    
    x = np.load('dataset/x_conc.npy')
    y = np.load('dataset/y_conc.npy')
    x_val = np.load('dataset/X_val.npy')
    y_val = np.load('dataset/y_val.npy')
    return x, y, x_val, y_val

def model_plot(model, file_name):
    for i in model:
        tf.keras.utils.plot_model(
        model,
        to_file=f"model_graphs/{file_name}.png",
        show_shapes=True,
        show_layer_names=True,
        expand_nested=True,
        dpi=96)
        
def plot_graph(model_history):
    list_plots = []
    for hist in model_history:
        graph = plt.figure()
        axis1 = graph.add_subplot(211)
        axis1.plot(hist.history['accuracy'])
        axis1.plot(hist.history['val_accuracy'])
        axis1.title.set_text('Model Accuracy')
        axis1.set_ylabel('Accuracy')
        axis1.set_xlabel('Epoch')
        axis1.legend(['Train', 'Test'], loc='upper left')
        
        axis2 = graph.add_subplot(212)
        axis2.plot(hist.history['loss'])
        axis2.plot(hist.history['val_loss'])
        axis2.title.set_text('Model Loss')
        axis2.set_ylabel('Loss')
        axis2.set_xlabel('Epoch')
        axis2.legend(['Train', 'Test'], loc='upper left')
        graph.tight_layout()
        plt.close()
        list_plots.append(graph)
    return list_plots

def plot_combine(plots, accs, loss, lrs, beta1, beta2, name):
    graph = plt.figure()
    axis1 = graph.add_subplot(211)
    axis1.plot(accs)
    axis1.title.set_text('General Model Accuracy')
    axis1.set_ylabel('Accuracy')
    axis1.set_xlabel('Folds')

    axis2 = graph.add_subplot(212)
    axis2.plot(loss)
    axis2.title.set_text('General Model Loss')
    axis2.set_ylabel('Loss')
    axis2.set_xlabel('Folds')
    
    graph.suptitle('General Metrics')
    graph.tight_layout()
    plt.close()
    
    graph1 = plt.figure()
    axis = graph1.add_subplot(311)
    axis.plot(lrs)
    axis.title.set_text('Learning Rate')
    axis.set_ylabel('Lr')
    axis.set_xlabel('Epochs')
    
    axis3 = graph1.add_subplot(312)
    axis3.plot(beta1)
    axis3.title.set_text('Beta 1')
    axis3.set_ylabel('beta1')
    axis3.set_xlabel('Epochs')
    
    axis4 = graph1.add_subplot(313)
    axis4.plot(beta2)
    axis4.title.set_text('Beta 2')
    axis4.set_ylabel('beta2')
    axis4.set_xlabel('Epochs')
    
    graph1.suptitle('Lr, Beta1, Beta2')
    graph1.tight_layout()
    plt.close()

    pdf = matplotlib.backends.backend_pdf.PdfPages(name)
    fold = 1
    for fig in plots:
        fig.suptitle(f'Fold {fold}')
        fig.tight_layout()
        plt.close()
        pdf.savefig(fig)
        fold += 1
    pdf.savefig(graph)
    pdf.savefig(graph1)
    pdf.close()

class GetLearningRate(Callback):
    def __init__(self, lr_array, beta1_array, beta2_array):
        self.lr_array = lr_array
        self.beta1_array = beta1_array
        self.beta2_array = beta2_array
    
    def on_epoch_end(self, epoch, logs=None):
        decay = self.model.optimizer.decay
        lr = self.model.optimizer.lr
        iters = self.model.optimizer.iterations
        beta_1 = self.model.optimizer.beta_1
        beta_2 = self.model.optimizer.beta_2
        lr = lr * (1. / (1. + decay * K.cast(iters, K.dtype(decay))))
        t = K.cast(iters, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(beta_2, t)) / (1. - K.pow(beta_1, t)))
        self.lr_array.append(lr_t)
        self.beta1_array.append(beta_1)
        self.beta2_array.append(beta_2)
        print(f'lr: {lr_t}, beta_1: {beta_1}, beta_2: {beta_2}')

def exp_decay(epoch):
    k = 0.01
    lrate = initial_lrate * exp(-k*epoch)
    return lrate

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr

# **Pre-Processing (Dont run)**

In [ ]:
# PATHS
root_path = 'pothole_detection'
dataset = 'dataset'
categories = ['pothole', 'no_pothole']
test_categories = ['test_pothole', 'test_no_pothole']
image_size = 120
data = []
data_test = []

In [ ]:
# LOAD TEST TRAIN VALUES
cnt = 0
for category in categories:
    folder = os.path.join(dataset, category)
    label = categories.index(category)
    for img in os.listdir(folder):
        try:
            img_path = os.path.join(folder, img)
            cnt+=1
            print(category, cnt, label)
            clear_output(wait = True)
            img_array = cv2.imread(img_path)
            img_array = cv2.resize(img_array, (image_size, image_size))
            data.append([img_array, label])
        except Exception as e:
            print(str(e))

for category in test_categories:
    folder = os.path.join(dataset, category)
    label = test_categories.index(category)
    for img in os.listdir(folder):
        try:
            img_path = os.path.join(folder, img)
            cnt+=1
            print(category, cnt, label)
            clear_output(wait = True)
            img_array = cv2.imread(img_path)
            img_array = cv2.resize(img_array, (image_size, image_size))
            data_test.append([img_array, label])
        except Exception as e:
            print(str(e))          

In [ ]:
# SPLIT TEST TRAIN VALUES
np.random.shuffle(data)
np.random.shuffle(data_test)

X_old = []
y_old = []
X_old_test = []
y_old_test = []

for features, labels in data:
    X_old.append(features)
    y_old.append(labels)

X_train = np.array(X_old)
y_train = np.array(y_old)

for features, labels in data_test:
    X_old_test.append(features)
    y_old_test.append(labels)

X_test = np.array(X_old_test)
y_test = np.array(y_old_test)

In [ ]:
# LOAD VALIDATION VALUES
root_path = ''
dataset = 'dataset'
categories = ['pothole', 'no_pothole']
image_size = 120

folder_path = 'validation'
data_new = []

cnt = 0
for category in categories:
    folder_new = os.path.join(dataset, folder_path, category)
    label_new = categories.index(category)
    for img_new in os.listdir(folder_new):
        img_path_new = os.path.join(folder_new, img_new)
        cnt+=1
        print(category, cnt, label_new)
        clear_output(wait = True)
        img_array_new = cv2.imread(img_path_new)
        img_array_new = cv2.resize(img_array_new, (image_size, image_size))
        data_new.append([img_array_new, label_new])

np.random.shuffle(data_new)
X_old = []
y_old = []

for features_new, labels_new in data_new:
    X_old.append(features_new)
    y_old.append(labels_new)

X_new = np.array(X_old)
y_new = np.array(y_old)

In [ ]:
# SAVE NP ARRAYS
# np.save('dataset/X_train', X_train)
# np.save('dataset/X_test', X_test)
# np.save('dataset/y_train', y_train)
# np.save('dataset/y_test', y_test)
np.save('dataset/X_val', X_new)
np.save('dataset/y_val', y_new)

x = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)
np.save('dataset/x_conc', x)
np.save('dataset/y_conc', y)

# **Load Data**

In [ ]:
# LOAD DATA
x = np.load('dataset/x_conc.npy')
y = np.load('dataset/y_conc.npy')
X_new = np.load('dataset/X_val.npy')
y_new = np.load('dataset/y_val.npy')
X_new = X_new / 255

# **Cross Validation**

In [ ]:
# DEFINE TRAIN_CROSS_VAL MODEL
def train_cross_val(input_layer_neurons_1_val, input_layer_neurons_2_val, input_layer_neurons_3_val, first_layer_neurons, mes):
    n_folds = 10
    pat = 3
    early_stopping = EarlyStopping(monitor='val_loss', patience=pat, verbose=1)
    model_checkpoint_best = ModelCheckpoint('checkpoints/' + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S") + '.h5', verbose=1, save_best_only=True)
    log_dir = "logs/fit/" + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S")
    logs = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    ### Model ###
    model = Sequential()

    model.add(Conv2D(input_layer_neurons_1_val, input_shape = x.shape[1:], kernel_size = 3, strides = 1, activation = 'relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size = 2, strides = 2))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(input_layer_neurons_2_val, kernel_size = 3, strides = 1, activation = 'relu'))
    model.add(MaxPooling2D(pool_size = 2, strides = 2))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(input_layer_neurons_3_val, kernel_size = 3, strides = 1, activation = 'relu'))
    model.add(MaxPooling2D(pool_size = 2, strides = 2))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Flatten())
x
    model.add(Dense(first_layer_neurons, activation = 'relu'))
    model.add(Dropout(0.4))

    model.add(Dense(2, activation = 'softmax'))

    model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
    acc_per_fold = []
    loss_per_fold = []
    model_histories = []
    
    print('training')
    print('Fold Num =', end=' ')
    for i in range(n_folds):
        print(i+1, end=' ')
        model_checkpoint = ModelCheckpoint('temp_checkpoint/' + "model_{epoch:02d}" + '_' + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S") + '.h5', verbose=0, save_best_only=False, monitor='val_loss')
        train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2,
                                                   random_state = np.random.randint(1,1000, 1)[0])
        
        train_x = train_x / 255
        test_x = test_x / 255
        
        history = model.fit(train_x, train_y, epochs=10, callbacks=[early_stopping, model_checkpoint, model_checkpoint_best], verbose=1, validation_split=(0.2))
        model_histories.append(history)
        
        scores = model.evaluate(test_x, test_y, verbose=0)
        
        acc_per_fold.append(scores[1])
        loss_per_fold.append(scores[0])
        
        tg_sendtext(f'Finished fold number {i+1} of {mes} with accuracy = {scores[1]*100}%')
        message_discord(f'Finished fold number {i+1} of {mes} with accuracy = {scores[1]*100}%')
        model.save('checkpoints/' + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S") + '.h5')
        check_highest_custom('temp_checkpoint/', 'checkpoints/', 97.4)
    print('Finished')
    return model_histories, acc_per_fold, loss_per_fold

# **Auto Trainer**

In [ ]:
# CREAT NEURON VALUES
input_layer_neurons_1 = [32, 64, 128]
input_layer_neurons_2 = [32, 64, 128]
input_layer_neurons_3 = [32, 64, 128]
first_layer_neurons = [128, 256, 512]
res = [[i, j, k, l]
       for i in input_layer_neurons_1
       for j in input_layer_neurons_2
       for k in input_layer_neurons_3
       for l in first_layer_neurons
      ]
new_res = []
for i in res:
    if i[0] > i[1]:
        new_res.append(i)
    elif i[1] > i[2]:
        new_res.append(i)
for i in new_res:
    res.remove(i)

In [ ]:
# AUTO TRAINER
maxx = 0
count = 1
maxx_acc = []
maxx_i = []
for i in res:
    print(count)
    neurons = [i[0], i[1], i[2], i[3]]
    print('Neurons =', i[0], i[1], i[2], i[3])
    message = 'model ' + str(count)
    tg_sendtext('Started training ' + message + ' with neurons = ' + str(neurons))
    message_discord('Started training ' + message + ' with neurons = ' + str(neurons))
    hist, accuracies, losses = train_cross_val(i[0], i[1], i[2], i[3], message)
    maxx, accuracy, maxx_acc, maxx_i = check_highest('checkpoints/', maxx, maxx_acc, maxx_i)
    plot_lists = plot_graph(hist)
    plot_combine(plot_lists, accuracies, losses, 'model_graphs/' + str(neurons) + '_' + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S") + '_' + str(round(accuracy, 2)) + '.pdf')
    message = message + ' ' + str([i[0], i[1], i[2], i[3]]) + ' with final accuracy = ' + str(accuracy)
    tg_sendtext('Finished training ' + message)
    message_discord('Finished training ' + message)
    count += 1
    print('------------------------------------------------------------------------')
    tg_sendtext('------------------------------------------------------------------------')
    message_discord('------------------------------------------------------------------------')
print('AHHHHHHH')
tg_sendtext(f'Training Completed | Highest Accuracy = {maxx}')
message_discord(f'Training Completed | Highest Accuracy = {maxx}')

# **Tester**

In [ ]:
# TESTER 
n_folds = 10
tg_sendtext(f'Training Model')
message_discord(f'Training Model')
pat = 3
lr_arr = []
beta1_arr = []
beta2_arr = []
early_stopping = EarlyStopping(monitor='val_loss', patience=pat, verbose=1)
model_checkpoint_best = ModelCheckpoint('checkpoints/' + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S") + '.h5', verbose=1, save_best_only=True)
log_dir = "logs/fit/" + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S")
logs = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
lr = GetLearningRate(lr_arr, beta1_arr, beta2_arr)

model = Sequential()

model.add(Conv2D(32, input_shape = x.shape[1:], kernel_size = 3, strides = 1, activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = 2, strides = 2))
model.add(Dropout(0.2071809982680866))

model.add(Conv2D(64, kernel_size = 3, strides = 1, activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = 2, strides = 2))
model.add(Dropout(0.04612987161018767))

model.add(Conv2D(128, kernel_size = 3, strides = 1, activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = 2, strides = 2))
model.add(Dropout(0.10471119809697471))

model.add(Flatten())

model.add(Dense(512, activation = 'relu', kernel_initializer = 'HeNormal'))
model.add(Dropout(0.28309374387487923))

model.add(Dense(2, activation = 'softmax'))

optimizer = tf.keras.optimizers.Nadam(lr=0.001)
lr_metric = get_lr_metric(optimizer)
model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy', lr_metric])

model_histories = []
acc_per_fold = []
loss_per_fold = []
factor = 0.7

for i in range(n_folds):
    print(f'Fold Number = {i+1}')
    factor = 0.7*((1.036311)**i)
    rlrop = ReduceLROnPlateau(monitor='val_loss', factor=factor, patience=3, verbose=1)
    model_checkpoint = ModelCheckpoint('temp_checkpoint/' + "model_{epoch:02d}" + '_' + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S") + '.h5', verbose=0, save_best_only=False, monitor='val_loss')
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state = np.random.randint(1,1000, 1)[0])

    train_x = train_x / 255
    test_x = test_x / 255  

    history = model.fit(train_x, train_y, epochs=10, callbacks=[early_stopping, model_checkpoint, model_checkpoint_best, logs, lr, rlrop], verbose=1, validation_split=(0.2))
    model_histories.append(history)

    scores = model.evaluate(test_x, test_y, verbose=0)

    acc_per_fold.append(scores[1])
    loss_per_fold.append(scores[0])

    print(f'Fold {i+1} Accuracy = {scores[1]*100}%')

    tg_sendtext(f'Finished fold number {i+1} with accuracy = {scores[1]*100}%')
    message_discord(f'Finished fold number {i+1} with accuracy = {scores[1]*100}%')
    model.save('checkpoints/' + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S") + '.h5')
    check_highest_rename('temp_checkpoint/', 'good_models/', 97.6, X_new, y_new)
score, accuracy = model.evaluate(X_new, y_new, verbose=0)
print('Accuracy:', accuracy)

learning_rates = lr.lr_array
beta_1 = lr.beta1_array
beta_2 = lr.beta2_arr
plot_lists = plot_graph(model_histories)
plot_combine(plot_lists, acc_per_fold, loss_per_fold, beta_1, beta_2, learning_rates,'model_graphs/' + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S") + '_' + str(round(accuracy, 2)) + '.pdf')

check_highest_rename('temp_checkpoint/', 'good_models/', 97.6, X_new, y_new)
check_highest_rename('checkpoints/', 'good_models/', 97.6, X_new, y_new)
check_highest_rename('saved_models/', 'good_models/', 97.6, X_new, y_new)
tg_sendtext(f'Finished training Model with accuracy = {accuracy}')
message_discord(f'Finished training Model with accuracy = {accuracy}')

In [ ]:
# TESTER
n_folds = 10
pat = 3
early_stopping = EarlyStopping(monitor='val_loss', patience=pat, verbose=1)
model_checkpoint_best = ModelCheckpoint('checkpoints/' + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S") + '.h5', verbose=1, save_best_only=True)
model_checkpoint = ModelCheckpoint('temp_checkpoint/' + "model_{epoch:02d}" + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S") + '.h5', verbose=0, save_best_only=False, monitor='val_loss')
rlrop = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=2)
log_dir = "logs/fit/" + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S")
logs = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
### Model ###
model = Sequential()

model.add(Conv2D(32, input_shape = X_train.shape[1:], kernel_size = 3, strides = 1, activation = 'relu', padding = 'same', kernel_initializer = 'random_uniform'))
model.add(MaxPooling2D(pool_size = 2, strides = 2))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(64, kernel_size = 3, strides = 1, activation = 'relu', kernel_initializer = 'random_uniform'))
model.add(MaxPooling2D(pool_size = 2, strides = 2))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Flatten())

model.add(Dense(128, activation = 'relu', kernel_initializer = 'random_uniform'))
model.add(Dropout(0.4))

model.add(Dense(2, activation = 'softmax', kernel_initializer = 'random_uniform'))

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=64, callbacks=[early_stopping, model_checkpoint, model_checkpoint_best, logs, rlrop], verbose=1, validation_data=(X_test, y_test))

scores = model.evaluate(X_new, y_new, verbose=0)

model.save('checkpoints/' + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S") + '.h5')
print('Finished')
# model.summary()

# **Hyperas**

In [ ]:
# CREATING HYPERAS MODEL

def create_model(x, y, x_val, y_val):
    n_folds = 10
    tg_sendtext(f'Training Model')
    message_discord(f'Training Model')
    pat = 3
    arr = []
    early_stopping = EarlyStopping(monitor='val_loss', patience=pat, verbose=0)
    model_checkpoint_best = ModelCheckpoint('checkpoints/' + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S") + '.h5', verbose=0, save_best_only=True)
    log_dir = "logs/fit/" + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S")
    logs = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    lr = GetLearningRate(arr)
    model = Sequential()

    model.add(Conv2D({{choice([32, 64])}}, input_shape = x.shape[1:], kernel_size = 3, strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = 2, strides = 2))
    model.add(Dropout({{uniform(0, 0.5)}}))


    model.add(Conv2D({{choice([32, 64])}}, kernel_size = 3, strides = 1, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = 2, strides = 2))
    model.add(Dropout({{uniform(0, 0.5)}}))


    model.add(Conv2D({{choice([32, 64, 128])}}, kernel_size = 3, strides = 1, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = 2, strides = 2))
    model.add(Dropout({{uniform(0, 0.5)}}))


    model.add(Flatten())

    model.add(Dense({{choice([128, 256, 512])}}, activation = 'relu'))
    model.add(Dropout({{uniform(0, 0.5)}}))

    model.add(Dense(2, activation = 'softmax'))
                    
    model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
    model_histories = []
    acc_per_fold = []
    loss_per_fold = []
    
    for i in range(n_folds):
        model_checkpoint = ModelCheckpoint('temp_checkpoint/' + "model_{epoch:02d}" + '_' + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S") + '.h5', verbose=0, save_best_only=False, monitor='val_loss')
        train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2,
                                                   random_state = np.random.randint(1,1000, 1)[0])
        
        train_x = train_x / 255
        test_x = test_x / 255
        
        history = model.fit(train_x, train_y, epochs=10, callbacks=[early_stopping, model_checkpoint, model_checkpoint_best, logs, lr], verbose=2, validation_split=(0.2))
        model_histories.append(history)
        
        scores = model.evaluate(test_x, test_y, verbose=0)
        
        acc_per_fold.append(scores[1])
        loss_per_fold.append(scores[0])
        
        print(f'Fold {i+1} Accuracy = {scores[1]*100}%')
        
        tg_sendtext(f'Finished fold number {i+1} with accuracy = {scores[1]*100}%')
        message_discord(f'Finished fold number {i+1} with accuracy = {scores[1]*100}%')
        model.save('checkpoints/' + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S") + '.h5')
        check_highest_rename('temp_checkpoint/', 'checkpoints/', 97, x_val, y_val)
    score, accuracy = model.evaluate(x_val, y_val, verbose=0)
    print('Accuracy:', accuracy)

    learning_rates = lr.array
    plot_lists = plot_graph(model_histories)
    plot_combine(plot_lists, acc_per_fold, loss_per_fold, learning_rates,'model_graphs/' + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S") + '_' + str(round(accuracy, 2)) + '.pdf')
    
    check_highest_rename('temp_checkpoint/', 'good_models/', 97, x_val, y_val)
    check_highest_rename('checkpoints/', 'good_models/', 97, x_val, y_val)
    check_highest_rename('saved_models/', 'good_models/', 97, x_val, y_val)
    tg_sendtext(f'Finished training Model with accuracy = {accuracy}')
    message_discord(f'Finished training Model with accuracy = {accuracy}')
    return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [ ]:
# HYPERAS OPTIM
best_run, best_model = optim.minimize(model=create_model, data=data, algo=tpe.suggest, max_evals=10, trials=Trials(), 
                                      notebook_name='pothole_detection_cross_validation_updated', 
                                      functions=[check_highest_rename, tg_sendtext, message_discord, check_accuracy, plot_graph, plot_combine])